In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
import re

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_excel("/content/drive/MyDrive/machine_learning/data/Brute/data_tweets2.xlsx")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/machine_learning/data/Brute/data_tweets2.xlsx'

In [ ]:
df = pd.read_excel("data_tweets2.xlsx")

In [ ]:
df

,text,label
0,$BYND - JPMorgan reels in expectations on Beyo...,0
1,$CCL $RCL - Nomura points to bookings weakness...,0
2,"$CX - Cemex cut at Credit Suisse, J.P. Morgan ...",0
3,$ESS: BTIG Research cuts to Neutral https://t....,0
4,$FNKO - Funko slides after Piper Jaffray PT cu...,0
...,...,...
9538,The Week's Gainers and Losers on the Stoxx Eur...,2
9539,Tupperware Brands among consumer gainers; Unil...,2
9540,vTv Therapeutics leads healthcare gainers; Myo...,2
9541,"WORK, XPO, PYX and AMKR among after hour movers",2


In [ ]:
df_url = df.copy()
df_url['text'] = df_url["text"].apply(lambda x: re.sub(r'http\S+', '<URL>', x))

In [ ]:
df_url

,text,label
0,$BYND - JPMorgan reels in expectations on Beyo...,0
1,$CCL $RCL - Nomura points to bookings weakness...,0
2,"$CX - Cemex cut at Credit Suisse, J.P. Morgan ...",0
3,$ESS: BTIG Research cuts to Neutral <URL>,0
4,$FNKO - Funko slides after Piper Jaffray PT cu...,0
...,...,...
9538,The Week's Gainers and Losers on the Stoxx Eur...,2
9539,Tupperware Brands among consumer gainers; Unil...,2
9540,vTv Therapeutics leads healthcare gainers; Myo...,2
9541,"WORK, XPO, PYX and AMKR among after hour movers",2


Remplacement des labels 2 (neutre) par 1 (positif), transformant la classification en binaire (0 = négatif, 1 = positif).

In [ ]:
df_url['label'] = df_url['label'].replace(2,1)

In [ ]:
df_url['label'].value_counts()

,count
label,
1,8101
0,1442


## Text Preprocessing

### Removing special characters

In [ ]:
df_url['text'] = df_url['text'].apply(lambda text: re.sub(r"[^a-zA-Z0-9\s]", "", text))
df_url

,text,label
0,BYND JPMorgan reels in expectations on Beyond...,0
1,CCL RCL Nomura points to bookings weakness at...,0
2,CX Cemex cut at Credit Suisse JP Morgan on we...,0
3,ESS BTIG Research cuts to Neutral URL,0
4,FNKO Funko slides after Piper Jaffray PT cut URL,0
...,...,...
9538,The Weeks Gainers and Losers on the Stoxx Euro...,1
9539,Tupperware Brands among consumer gainers Unile...,1
9540,vTv Therapeutics leads healthcare gainers Myom...,1
9541,WORK XPO PYX and AMKR among after hour movers,1


### Lemmatization

In [ ]:
df_url['text']= df_url['text'].str.lower()

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
def process_text(texts):
  docs = list(nlp.pipe(texts))
  return [" ".join([token.lemma_ for token in doc]) for doc in docs]

In [ ]:
df_url['lemma'] = process_text(df_url["text"].to_list())

In [ ]:
df_url

,text,label,lemma
0,bynd jpmorgan reels in expectations on beyond...,0,bynd jpmorgan reel in expectation on beyond ...
1,ccl rcl nomura points to bookings weakness at...,0,ccl rcl nomura point to booking weakness at ...
2,cx cemex cut at credit suisse jp morgan on we...,0,cx cemex cut at credit suisse jp morgan on w...
3,ess btig research cuts to neutral url,0,ess btig research cut to neutral url
4,fnko funko slides after piper jaffray pt cut url,0,fnko funko slide after piper jaffray pt cut url
...,...,...,...
9538,the weeks gainers and losers on the stoxx euro...,1,the week gainer and loser on the stoxx europe ...
9539,tupperware brands among consumer gainers unile...,1,tupperware brand among consumer gainer unileve...
9540,vtv therapeutics leads healthcare gainers myom...,1,vtv therapeutic lead healthcare gainer myomo a...
9541,work xpo pyx and amkr among after hour movers,1,work xpo pyx and amkr among after hour mover


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialiser le vectorizer TF-IDF
tfidf_vectorizer = TfidfVectorizer()

# Transformer les tweets en vecteurs TF-IDF
X_tfidf = tfidf_vectorizer.fit_transform(df_url["lemma"])

# Afficher la matrice résultante
print(X_tfidf[:1])  # (n_samples, n_features)


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9 stored elements and shape (1, 14812)>
  Coords	Values
  (0, 2938)	0.4680746258731047
  (0, 7652)	0.3701167358184351
  (0, 11259)	0.4322839348014065
  (0, 7128)	0.14663515316959105
  (0, 5462)	0.3330510403987986
  (0, 9910)	0.15323893248322837
  (0, 2435)	0.3727652677680154
  (0, 8645)	0.3923602343152121
  (0, 13994)	0.09078994318223606


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report


In [ ]:
x = X_tfidf
y = df_url['label']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
# Initialize and train the model
log_reg = LogisticRegression(class_weight='balanced',max_iter=500) # max_tier 500 is for enough iteration so that the model converge
log_reg.fit(x_train, y_train)


LogisticRegression(class_weight='balanced', max_iter=500)

In [ ]:
y_pred=log_reg.predict(x_test)

In [ ]:
# Print accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Print detailed classification report (Precision, Recall, F1-score)
print("Logistic regression:\n", classification_report(y_test, y_pred))


Accuracy: 0.8774
Logistic regression:
               precision    recall  f1-score   support

           0       0.59      0.70      0.64       300
           1       0.94      0.91      0.93      1609

    accuracy                           0.88      1909
   macro avg       0.77      0.81      0.78      1909
weighted avg       0.89      0.88      0.88      1909



In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(x_train, y_train)

y_pred_knn = knn.predict(x_test)
accuracy = accuracy_score(y_test, y_pred_knn)
print(f"Accuracy: {accuracy:.4f}")
print("KNN Performance:\n", classification_report(y_test, y_pred_knn))


Accuracy: 0.8690
KNN Performance:
               precision    recall  f1-score   support

           0       0.72      0.28      0.40       300
           1       0.88      0.98      0.93      1609

    accuracy                           0.87      1909
   macro avg       0.80      0.63      0.66      1909
weighted avg       0.85      0.87      0.84      1909



In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(x_train, y_train)

y_pred_rf = rf.predict(x_test)
accuracy = accuracy_score(y_test, y_pred_rf)
print(f"Accuracy: {accuracy:.4f}")
print("Random Forest Performance:\n", classification_report(y_test, y_pred_rf))

Accuracy: 0.8869
Random Forest Performance:
               precision    recall  f1-score   support

           0       0.85      0.34      0.49       300
           1       0.89      0.99      0.94      1609

    accuracy                           0.89      1909
   macro avg       0.87      0.66      0.71      1909
weighted avg       0.88      0.89      0.87      1909



In [ ]:
from sklearn.neural_network import MLPClassifier

# Define a simple MLP model
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)

# Train the model
mlp.fit(x_train, y_train)

# Predict on the test set
y_pred_mlp = mlp.predict(x_test)

# Print accuracy and classification report
accuracy = accuracy_score(y_test, y_pred_mlp)
print(f"Neural Network Accuracy: {accuracy:.4f}")
print("Neural Network Performance:\n", classification_report(y_test, y_pred_mlp))


Neural Network Accuracy: 0.8858
Neural Network Performance:
               precision    recall  f1-score   support

           0       0.66      0.56      0.61       300
           1       0.92      0.95      0.93      1609

    accuracy                           0.89      1909
   macro avg       0.79      0.75      0.77      1909
weighted avg       0.88      0.89      0.88      1909



In [ ]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
nb.fit(x_train, y_train),

y_pred_nb = nb.predict(x_test)

accuracy_nb = accuracy_score(y_test, y_pred_nb)
print(f"Accuracy: {accuracy_nb:.4f}")
print("Naive Bayes Performance : \n", classification_report(y_test, y_pred_nb))

Accuracy: 0.8481
Naive Bayes Performance : 
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       288
           1       0.85      1.00      0.92      1621

    accuracy                           0.85      1909
   macro avg       0.42      0.50      0.46      1909
weighted avg       0.72      0.85      0.78      1909



In [ ]:
from sklearn.svm import SVC

svm = SVC(kernel='linear',class_weight= 'balanced')
svm.fit(x_train, y_train)

y_pred_svm = svm.predict(x_test)

accuracy = accuracy_score(y_test, y_pred_svm)
print(f"Accuracy: {accuracy:.4f}")
print(f"SVM Performance : \n", classification_report(y_test, y_pred_svm))

Accuracy: 0.8748
SVM Performance : 
               precision    recall  f1-score   support

           0       0.59      0.65      0.62       300
           1       0.93      0.92      0.93      1609

    accuracy                           0.87      1909
   macro avg       0.76      0.78      0.77      1909
weighted avg       0.88      0.87      0.88      1909



# PART 2
In this part we will try to optimize the Naive Bayes model since it did so poorly by varying X

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_bow = CountVectorizer(ngram_range=(1,2), max_features=3000)
X_bow = vectorizer_bow.fit_transform(df_url["lemma"])

Here we changed from an TF-IDF method of

In [ ]:
X_train_bow, X_test_bow, y_train, y_test = train_test_split(
    X_bow, df_url["label"], test_size=0.2, stratify=df_url["label"], random_state=42
)

# Initialize and train the Naive Bayes model
nb_model_bow = MultinomialNB()
nb_model_bow.fit(X_train_bow, y_train)

# Predict on the test set
y_pred_nb_bow = nb_model_bow.predict(X_test_bow)

# Evaluate performance
print(accuracy_score(y_test, y_pred_nb_bow))
print(classification_report(y_test, y_pred_nb_bow))

0.8737558931377685
              precision    recall  f1-score   support

           0       0.58      0.61      0.59       288
           1       0.93      0.92      0.93      1621

    accuracy                           0.87      1909
   macro avg       0.75      0.77      0.76      1909
weighted avg       0.88      0.87      0.88      1909



In [ ]:
accuracy_nb = accuracy_score(y_test, y_pred_nb)
print(f"Accuracy: {accuracy_nb:.4f}")
print("Naive Bayes Performance : \n", classification_report(y_test, y_pred_nb))

Accuracy: 0.8481
Naive Bayes Performance : 
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       288
           1       0.85      1.00      0.92      1621

    accuracy                           0.85      1909
   macro avg       0.42      0.50      0.46      1909
weighted avg       0.72      0.85      0.78      1909

